# Database parser  <br>
based on Tine's script

In [10]:
import pandas as pd
from collections import defaultdict
import mysql.connector
import glob
import os
import csv
import numpy as np
pd.set_option('display.max_columns', 10)

In [11]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',
                               database='expression_atlas_cells2')
mycursor = conn.cursor(buffered=True)

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


1. Only projects with confirmed cell annotation will be accessible in the database. These can be found in file "annotation_excel_use.xlsx"

In [12]:
manual_meta = pd.read_csv("/home/compomics/Sam/git/python/Sam/Tine/annotation_excel_tine.csv", sep = ";")
manual_meta = manual_meta.fillna("Not found")
manual_meta.head()

,PXD,RAW,cell,treated,comment
0,PXD006421,FASP_HL60_0h1.raw.xz,HL60,ATRA,"hour states 'hours after ATRA treatment', 5 te..."
1,PXD006421,FASP_HL60_0h2.raw.xz,HL60,ATRA,Not found
2,PXD006421,FASP_HL60_0h3.raw.xz,HL60,ATRA,Not found
3,PXD006421,FASP_HL60_0h4.raw.xz,HL60,ATRA,Not found
4,PXD006421,FASP_HL60_0h5.raw.xz,HL60,ATRA,Not found


In [13]:
projects = list(manual_meta.PXD.unique())
print(len(projects))



49


In [14]:
manual_meta.RAW = manual_meta.RAW.apply(lambda x: x.split(".")[0])
manual_meta.tail()

,PXD,RAW,cell,treated,comment
1971,PXD008996,A431_control_3_2,A449,No,Shotgun
1972,PXD008996,A431_1uM_DON_1_2,A450,"DON (Deoxynivalenol, vomitoxin)",Shotgun
1973,PXD008996,A431_01uM_DON_2_1,A451,"DON (Deoxynivalenol, vomitoxin)",Shotgun
1974,PXD008996,A431_1uM_DON_2_1,A452,"DON (Deoxynivalenol, vomitoxin)",Shotgun
1975,PXD008996,A431_10uM_DON_3_2,A453,"DON (Deoxynivalenol, vomitoxin)",Shotgun


In [6]:
manual_meta.shape

(1976, 5)

2. the metadata file from the PRIDE preprocessing will be used to populate the "project" table
- project_id (automatically generated)
- PXD_accession
- experiment_type
- instruments
- keywords

In [15]:
meta = pd.read_csv("/home/compomics/Sam/git/python/master_thesis/Metadata/metadata_ionbot_update_0401.csv", sep = ";")
meta = meta.iloc[:, 0:15]
meta = meta.astype(str)
meta.head()

,accession,url,title,projectDescription,species,...,quantificationMethods,keywords,dataProcessingProtocol,submissionDate,references
0,PXD003411,https://www.ebi.ac.uk/pride/ws/archive/project...,Contribution of primary human fibroblasts and ...,While the most important players of inflammati...,"[""Homo sapiens (Human)""]",...,nan,"Cell type-specific inflammatory response, hall...",Proteome Discoverer 1.4 (Thermo Fisher Scienti...,5/01/2016,"[[""DOI:null"",""PMID:27025457""]]"
1,PXD001000,https://www.ebi.ac.uk/pride/ws/archive/project...,CPTAC System Suitability (CompRef) Study: Phos...,"The objective of the ""System Suitability (Comp...","[""Homo sapiens (Human)"",""Mus musculus (Mouse)""]",...,"[""iTRAQ""]",CPTAC CPTAC-CompRef CPTAC-PNNL,Processed using the CPTAC Common Data Analysis...,23/05/2014,nan
2,PXD001838,https://www.ebi.ac.uk/pride/ws/archive/project...,Proteomics of EBV-infected lytic and refractor...,Most humans are infected with Epstein-Barr vir...,"[""Homo sapiens (Human)""]",...,"[""Spectrum counting""]","Epstein-Barr virus, lytic activation, proteomi...",Tandem mass spectra were extracted from raw fi...,27/02/2015,"[[""DOI:10.1128/JVI.00121-15"",""PMID:25717101""]]"
3,PXD009074,https://www.ebi.ac.uk/pride/ws/archive/project...,Human T-cells EVs LC-MSMS - Extracellular Ves...,As it is likely that different stimuli promote...,"[""Homo sapiens (Human)""]",...,nan,"Apoptosis, EVs, microvesicles, exosomes, LC-MSMS",5. Two dimensional difference gel electrophore...,28/02/2018,"[[""DOI:10.3389/fimmu.2018.00534"",""PMID:2959978..."
4,PXD003943,https://www.ebi.ac.uk/pride/ws/archive/project...,Host targets of Citrobacter secreted effectors,Identification of human proteins targeted by C...,"[""Homo sapiens (Human)""]",...,nan,"effectors, host targets, type 3 secretion system",Data was searched with MSGF+ using PNNL's DMS ...,12/04/2016,nan


In [20]:
def build_project_table(meta_df, list_of_pxds):
    count = 0
    meta_df = meta_df[meta_df['accession'].isin(list_of_pxds)]
    meta_df = meta_df[['accession', 'experimentTypes', 'instrumentNames', 'keywords', 'references']]
    meta_df = meta_df.astype(str)
    meta_tuples = list(meta_df.to_records(index=False)) #a list of tuples is easily iteratible and easy to store in the database
    for i in meta_tuples:
        count += 1
        project = "INSERT INTO project(PXD_accession, experiment_type, instrument, keywords, ref) VALUES (%s, %s, %s, %s, %s)"
        i = list(i)
        mycursor.execute(project, i)
        conn.commit()
        print(count)

In [21]:
build_project_table(meta, projects)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


3. Prior to linking the assay to the cell_line, the cell table must be made using all the cell lines and cell types that are present in the annotation_excel_use.csv file
The cell table needs:
- cell_line
- disease
- treatment
- tissue_type

In [22]:
manual_meta

,PXD,RAW,cell,treated,comment
0,PXD006421,FASP_HL60_0h1,HL60,ATRA,"hour states 'hours after ATRA treatment', 5 te..."
1,PXD006421,FASP_HL60_0h2,HL60,ATRA,Not found
2,PXD006421,FASP_HL60_0h3,HL60,ATRA,Not found
3,PXD006421,FASP_HL60_0h4,HL60,ATRA,Not found
4,PXD006421,FASP_HL60_0h5,HL60,ATRA,Not found
...,...,...,...,...,...
1971,PXD008996,A431_control_3_2,A449,No,Shotgun
1972,PXD008996,A431_1uM_DON_1_2,A450,"DON (Deoxynivalenol, vomitoxin)",Shotgun
1973,PXD008996,A431_01uM_DON_2_1,A451,"DON (Deoxynivalenol, vomitoxin)",Shotgun
1974,PXD008996,A431_1uM_DON_2_1,A452,"DON (Deoxynivalenol, vomitoxin)",Shotgun


In [39]:
manual_cell_lines = manual_meta[["cell", 'treated']]
manual_cell_lines = manual_cell_lines.drop_duplicates()

In [40]:
manual_cell_lines

,cell,treated
0,HL60,ATRA
25,LAN5,No
28,LAN5,recombinant Aβ42
33,MCF7,No
77,MCF7,GDC-0941
...,...,...
1971,A449,No
1972,A450,"DON (Deoxynivalenol, vomitoxin)"
1973,A451,"DON (Deoxynivalenol, vomitoxin)"
1974,A452,"DON (Deoxynivalenol, vomitoxin)"


In [41]:
def build_cell_table(cell_df):
    count = 0
    cell_tuples = list(cell_df.to_records(index=False)) #a list of tuples is easily iteratible and easy to store in the database
    for i in cell_tuples:
        count += 1
        cell = "INSERT INTO cell(cell_line, treated) VALUES (%s, %s)"
        i = list(i)
        mycursor.execute(cell, i)
        conn.commit()
    print(count)

In [42]:
build_cell_table(manual_cell_lines)

200


4. Load the **modification table** from the file Tables/modifications.csv into the database
--> not done yet

In [25]:
pass

5. Load the **protein** table --> not done yet <br>

The protein table should be done afterwards based on the proteins found in the assays (merge with uniprot csv file on the uniprotID)

In [43]:
#Use uniprot files to list uniprot_id, length, description and sequence
tine_file_path = "/home/compomics/Sam/git/python/Sam/Database/"
uniprot = pd.read_csv(tine_file_path + "Uniprot.csv", sep = ",")

seq = pd.read_csv(tine_file_path + "Uniprot_sequences.csv", sep = ";")

uni_seq = pd.merge(uniprot, seq, on = "uniprot_id", how = "left")
uni_seq = uni_seq.drop_duplicates()

In [44]:
#Link and save all the corresponding uniprot_ids in the database with the length, description and sequence of the uniprot files
query = "SELECT DISTINCT(uniprot_id) from protein"
mysqlData = pd.read_sql_query(query, conn)

print(mysqlData.shape)
mysqlData['description'] = np.nan
mysqlData['length'] = np.nan
mysqlData['sequence'] = np.nan

total_ids = pd.merge(uni_seq, mysqlData, how = "left", on = "uniprot_id")
print(total_ids.shape)

total_ids = total_ids.drop(columns=['description_y', 'length_y', 'sequence_y'])
total_ids = total_ids.rename(columns={'description_x' : 'description', 'length_x' : 'length', 'sequence_x': 'sequence'})

total_ids.to_csv("protein_table_uniprot.csv", sep = ",", index = False)
total_ids.head()

(0, 1)
(20385, 7)


,uniprot_id,description,length,sequence
0,Q8N7X0,Androglobin (Calpain-7-like protein),1667,MASKQTKKKEVHRINSAHGSDKSKDFYPFGSNVQSGSTEQKKGKFP...
1,Q5T1N1,Protein AKNAD1,836,MDEADFSEHTTYKQEDLPYDGDLSQIKIGNDYSFTSKKDGLEVLNQ...
2,Q92667,"A-kinase anchor protein 1, mitochondrial (A-ki...",903,MAIQFRSLFPLALPGMLALLGWWWFFSRKKGHVSSHDEQQVEAGAV...
3,Q5VUY0,Arylacetamide deacetylase-like 3 (EC 3.1.1.-),407,MWDLALIFLAAACVFSLGVTLWVICSHFFTVHIPAAVGHPVKLRVL...
4,P62736,"Actin, aortic smooth muscle (Alpha-actin-2) (C...",377,MCEEEDSTALVCDNGSGLCKAGFAGDDAPRAVFPSIVGRPRHQGVM...


In [45]:
uniprot_df = pd.read_csv("protein_table_uniprot.csv", sep = ",")
non_human = """ADH1_YEAST,,,
ALBU_BOVIN,,,
ALDOA_RABIT,,,
BGAL_ECOLI,,,
CAH2_BOVIN,,,
CAS1_BOVIN,,,
CAS2_BOVIN,,,
CASB_BOVIN,,,
CASK_BOVIN,,,
CYC_HORSE,,,
DHE3_BOVIN,,,
GAG_SCVLA,,,
GFP_AEQVI,,,
K1C15_SHEEP,,,
K1M1_SHEEP,,,
K1M2_SHEEP,,,
K2M1_SHEEP,,,
K2M2_SHEEP,,,
K2M3_SHEEP,,,
KRA3_SHEEP,,,
KRA61_SHEEP,,,
LALBA_BOVIN,,,
LYSC_CHICK,,,
LYSC_LYSEN,,,
MYG_HORSE,,,
REF_HEVBR,,,
SRPP_HEVBR,,,
TRY1_BOVIN,,,
TRYP_PIG,,,
Q9BZD3""".split(",,,\n")

#TODO Q9BZD3 is omitted because no sequence was found. Resolve this.

def build_protein_table(uniprot_df):
    count = 0
    uniprot_tuples = list(uniprot_df.to_records(index=False)) #a list of tuples is easily iteratible and easy to store in the database
    for i in uniprot_tuples:
        count += 1
        query = "UPDATE protein SET description = %s, length = %s, sequence = %s WHERE uniprot_id = %s"
        uni_id, desc, length, sequence = list(i)
        i = [desc, float(length), sequence, uni_id]
        mycursor.execute(query, i)
        conn.commit()
    print(count)

check_na = uniprot_df[uniprot_df.isna().any(axis=1)]
uniprot_df = uniprot_df[~uniprot_df['uniprot_id'].isin(non_human)]    
print(uniprot_df.tail())

      uniprot_id                                        description  length  \
20380     Q8N895  Zinc finger protein 366 (Dendritic cell-specif...     744   
20381     Q9UK55  Protein Z-dependent protease inhibitor (PZ-dep...     444   
20382     Q96MX3   Zinc finger protein 48 (Zinc finger protein 553)     618   
20383     A6NGD5  Zinc finger and SCAN domain-containing protein...     496   
20384     Q9Y4E5  E3 SUMO-protein ligase ZNF451 (EC 2.3.2.-) (Co...    1061   

                                                sequence  
20380  MQKEMKMIKDEDVHFDLAVKKTPSFPHCLQPVASRGKAPQRHPFPE...  
20381  MKVVPSLLLSVLLAQVWLVPGLAPSPQSPETPAPQNQTSRVVQAPK...  
20382  MERAVEPWGPDLHRPEEREPQRGARTGLGSENVISQPNEFEHTPQE...  
20383  MAANCTSSWSLGESCNSPGSEPPQSMPSPATQLGNHDSDPETCHVN...  
20384  MGDPGSEIIESVPPAGPEASESTTDENEDDIQFVSEGPLRPVLEYI...  


In [46]:
uniprot_df = uniprot_df.replace(np.nan,'NaN')

In [47]:
build_protein_table(uniprot_df)

20384


6. The *annotation_excel_use2.csv* file will be used to build the **assay table**:
- assay_id: auto_incremented
- project_id from the project_table, linked to the PXD from the file
- filename: assay name
<br>

Simultaneously, the link to the cell line must be made.

In [55]:
def build_assay_cell_table(assay_df):
    count = 0
    assay_tuples = list(assay_df.to_records(index = False))
    for i in assay_tuples:
        (accession, filename, cell_line, treated, comment) = i
        
        filename = filename.split(".")[0]
        
        #select project_id
        mycursor.execute("SELECT project_id FROM project where PXD_accession = %s", (accession,))
        projectID_tup = mycursor.fetchone()
        (projectID,) = projectID_tup
        #insert into assay table
        assay = "insert into assay(project_id, filename) VALUES(%s, %s)"
        projectID_filename = (projectID, filename)
        mycursor.execute(assay, projectID_filename)
        conn.commit()
        #store this automatically generated assay ID for the cell_to_assay table
        assayID = mycursor.lastrowid
        #select cellID
        mycursor.execute("SELECT cell_id FROM cell WHERE cell_line = %s AND treated = %s", (cell_line, treated))
        cellID_tup = mycursor.fetchone()
        (cellID,) = cellID_tup
        #insert cellID and assayID in cell_to_assay
        cell_to_assay = "INSERT INTO cell_to_assay(assay_id, cell_id) VALUES(%s, %s)"
        assayID_cellID = (assayID, cellID)
        mycursor.execute(cell_to_assay, assayID_cellID)
        conn.commit()
        count += 1
    print(count)

In [56]:
build_assay_cell_table(manual_meta)

1976


7. The following step is using the ionbot information to fill in the **peptide_to_assay**, **peptide** and **peptide_to_modifications** tables.

In [57]:
def ionbot_parse(file):
    df = pd.read_csv(file, sep=',')
    # best_psm is equal to 1
    df = df.loc[df['best_psm'] == 1]
    #  q-value-best <= 0.01
    df = df.loc[df['q_value'] <= 0.01]
    # DB column needs to contain 'T' (otherwise decoy hit) +  extra check: only retain swissprot entries (start with sp)
    df = df.loc[df['DB'] == 'T']
    df_validated = df[df['proteins'].astype(str).str.startswith('sp')]
    # remove peptides that are not uniquely identified and are linked to multiple proteins = containing || in proteins
    x = '||'
    # regex is False otherwise it also detects a single | which is in every protein present
    df_validated = df_validated[~df_validated['proteins'].str.contains(x, regex=False)]
    # check not all entries were removed
    if df_validated.empty:
        return False

    # modifications can be linked to unimod id: peptide_modifications: unimod ID vs peptide

    # calculte the spectral counts from each peptide: dict: count
    peptides = df_validated['matched_peptide'].tolist()
    spectral_counts = defaultdict(int)
    for pep in peptides:
        spectral_counts[pep] += 1
    spectral_counts = dict(sorted(spectral_counts.items(), key=lambda item: item[1], reverse=True))
    print('parsed check')
    return df_validated, spectral_counts

In [58]:
def ionbot_store(file, filename):
    #check if the assay isn't already in the assay table
    filename = filename.split('/')[-1].split('.')[0]

    mycursor.execute("SELECT assay_id FROM assay WHERE filename = %s", (filename,))
    assayIDtup = mycursor.fetchone()
    if assayIDtup is None:
        print('{} is not in assays'.format(filename))
        return
    (assayID,) = assayIDtup
    parser = ionbot_parse(file)
    if parser is False:
        return
    df_validated, spectral_counts = parser

    # use the pandeylines in assay format
    # pandeylines resulted in a pd dataframe with all the proteins and sequences of validated peptides
    # loop over all rows/peptides present in the file (pandey_validated dataframe)
    df_validated_store = df_validated[['proteins', 'matched_peptide', 'modifications']]
    df_validated_tuples = [tuple(x) for x in df_validated_store.to_numpy()]
    for t in df_validated_tuples:
        protID = (t[0])
        pepseq = tuple((t[1],))
        mod = list((t[2],))

        # peptide storage - peptide ID
        sequence = "INSERT INTO peptide(peptide_sequence) VALUES (%s) " \
                    "ON DUPLICATE KEY UPDATE peptide_sequence=peptide_sequence"
        mycursor.execute(sequence, pepseq)
        conn.commit()

        # retrieve peptide_id, do not generate a new id each time!
        mycursor.execute("SELECT peptide_id FROM peptide WHERE peptide_sequence = %s", (pepseq))
        pepIDtup = mycursor.fetchone()
        (pepID,) = pepIDtup

        # link uniProtID = protein in assay to peptide
        proteinID = "INSERT INTO protein(uniprot_id) VALUES (%s) ON DUPLICATE KEY UPDATE uniprot_id=uniprot_id"
        uniprotID = (protID.split('|')[1],)
        mycursor.execute(proteinID, uniprotID)
        conn.commit()

        # relation peptide to protein
        pepToProt = "INSERT INTO peptide_to_protein(uniprot_id, peptide_id) VALUES (%s,%s) " \
                    "ON DUPLICATE KEY UPDATE peptide_id=peptide_id, uniprot_id=uniprot_id"
        uniprotIDstr = ''.join(uniprotID)
        uniprotID_peptideID = (uniprotIDstr, pepID)
        mycursor.execute(pepToProt, uniprotID_peptideID)
        conn.commit()

        for i in mod:
            if pd.isnull(i):
                break
            else:
                # retrieve modification id, peptide id is present
                location = (i.split('|')[0],)
                id = (i[i.find("[")+1:i.find("]")],)

                #retrieve modID
                mycursor.execute("SELECT mod_id FROM modifications WHERE mod_id = %s", (id))
                modIDtup = mycursor.fetchone()
                if modIDtup is None:
                    break
                (modID,) = modIDtup
                # relation peptide_to_modification
                peptoMod = "INSERT INTO peptide_modifications(peptide_id, location, mod_id, assay_id) VALUES (%s, %s, %s, %s)" \
                            "ON DUPLICATE KEY UPDATE peptide_id = peptide_id, mod_id = mod_id, assay_id=assay_id"
                peptoModvalues = pepIDtup + location + modIDtup + assayIDtup
                mycursor.execute(peptoMod, peptoModvalues)
                conn.commit()

        # spectral count for peptide
        count = float('inf')
        for k, v in spectral_counts.items():
            if k == (''.join(pepseq)):
                count = v
                break
        peptideToAssay = "INSERT INTO peptide_to_assay(peptide_id, assay_id, quantification) VALUES (%s, %s, %s) " \
                            "ON DUPLICATE KEY UPDATE peptide_id=peptide_id, assay_id=assay_id"
        peptideID_assayID_count = (pepID, assayID, count)
        mycursor.execute(peptideToAssay, peptideID_assayID_count)
        conn.commit()
    print('{} was stored'.format(filename))

In [59]:
def find_ionbot_files53(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/mounts/conode53/*/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [60]:
def find_ionbot_files54(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/mounts/conode54/*/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.2/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [61]:
def find_ionbot_files55(projects):
    number_of_files = 0
    for pxd in projects:
        path = '/home/compomics/mounts/conode55/pride/PRIDE_DATA/' + str(pxd) + '/IONBOT_v0.6.3/*.mgf.ionbot.csv'
        read_files = []
        for file in glob.glob(path):
            number_of_files += 1
            if file not in read_files:
                read_files.append(file)
                if os.path.getsize(file) != 0:
                    filename = str(file)
                    ionbot_store(file, filename)
    print(number_of_files)

In [62]:
find_ionbot_files53(projects)

parsed check
FASP_HL60_96h4 was stored
parsed check
FASP_HL60_24h5 was stored
parsed check
FASP_HL60_0h2 was stored
parsed check
FASP_HL60_96h2 was stored
parsed check
FASP_HL60_24h4 was stored
parsed check
FASP_HL60_48h3 was stored
parsed check
FASP_HL60_48h1 was stored
parsed check
FASP_HL60_96h3 was stored
parsed check
FASP_HL60_0h1 was stored
parsed check
FASP_HL60_24h3 was stored
parsed check
FASP_HL60_3h2 was stored
parsed check
FASP_HL60_96h1 was stored
parsed check
FASP_HL60_24h2 was stored
parsed check
FASP_HL60_96h5 was stored
parsed check
FASP_HL60_0h4 was stored
parsed check
FASP_HL60_3h1 was stored
parsed check
FASP_HL60_48h5 was stored
parsed check
FASP_HL60_3h4 was stored
parsed check
FASP_HL60_48h4 was stored
parsed check
FASP_HL60_24h1 was stored
parsed check
FASP_HL60_0h3 was stored
parsed check
FASP_HL60_3h3 was stored
parsed check
FASP_HL60_3h5 was stored
parsed check
FASP_HL60_0h5 was stored
parsed check
FASP_HL60_48h2 was stored
parsed check
FASP_A_01 was stored
p

In [63]:
find_ionbot_files54(projects)

parsed check
20160205_PGM_K052_SCX_RP_18 was stored
parsed check
160116_K052_OffLRP_RP_f15 was stored
parsed check
160116_K052_OffLRP_RP_f18 was stored
parsed check
20160205_PGM_K052_SCX_RP_04 was stored
parsed check
20160205_PGM_K052_SCX_RP_19 was stored
parsed check
160116_K052_OffLRP_RP_f08 was stored
parsed check
20160205_PGM_K052_SCX_RP_11 was stored
parsed check
160116_K052_OffLRP_RP_f02 was stored
parsed check
20160205_PGM_K052_SCX_RP_17 was stored
parsed check
20160205_PGM_K052_SCX_RP_07 was stored
parsed check
160116_K052_OffLRP_RP_f04 was stored
parsed check
20160205_PGM_K052_SCX_RP_15 was stored
parsed check
20160205_PGM_K052_SCX_RP_10 was stored
parsed check
20160205_PGM_K052_SCX_RP_03 was stored
parsed check
20160205_PGM_K052_SCX_RP_08 was stored
parsed check
20160205_PGM_K052_SCX_RP_05 was stored
parsed check
20160205_PGM_K052_SCX_RP_06 was stored
parsed check
20160205_PGM_K052_SCX_RP_16 was stored
parsed check
160116_K052_OffLRP_RP_f22 was stored
parsed check
20160205_PG

In [64]:
find_ionbot_files55(projects)

parsed check
20181130_PAK451-b2KO3 was stored
parsed check
20181213_PAK451_b2S_C3 was stored
parsed check
20181213_PAK451_b2S_KO2 was stored
parsed check
20181130_PAK451-b1C3 was stored
parsed check
20181130_PAK451-b1KO2 was stored
parsed check
20181130_PAK451-b2C2 was stored
parsed check
20181130_PAK451-b2KO1 was stored
parsed check
20181130_PAK451-b2KO2 was stored
parsed check
20181213_PAK451_b2S_C1 was stored
parsed check
20181213_PAK451_b1S_KO2 was stored
parsed check
20181130_PAK451-b1C2 was stored
parsed check
20181213_PAK451_b1S_KO3_20181218125716 was stored
parsed check
20181130_PAK451-b2C1 was stored
parsed check
20181213_PAK451_b2S_KO1 was stored
parsed check
20181213_PAK451_b1S_C1 was stored
parsed check
20181213_PAK451_b2S_KO3 was stored
parsed check
20181130_PAK451-b1C1 was stored
parsed check
20181213_PAK451_b1S_C2_20181216000514 was stored
parsed check
20181213_PAK451_b1S_C3 was stored
parsed check
20181130_PAK451-b1KO3 was stored
parsed check
20181130_PAK451-b1KO1 was s